In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Grid
nx = 101
xl, xr = 0., 101.
dx = (xr - xl) / nx

# Courant number
nu = 0.25

# Advection speed(constant)
c = 1.

# Scheme
# 'ftcs', 'lf','lw','upwind'
scheme = 'upwind'

# Output dir
out_dir = 'upwind'

nghost = 1
nxt = nx + 2 * nghost
dt = nu * dx / c
t = 0.
t_end = 10
nstep = 0
n_output = 2

In [3]:
# Initialnl grid
x = np.linspace(xl-1, xr+1, nxt)

In [4]:
# Initial conditions
u = np.where(x<50.,1.,0.)

In [5]:
# Time integrtion loop
while t < t_end:
    
    # Set boundary condition
    u[0]=u[1]
    u[nxt-1]=u[nxt - 2]
    
    # Calucurate new solution. Create left,right, and inner arrays to construct central differences
    u_right= u[nghost+1:]
    u_left = u[:-nghost-1]
    u_inner = u[nghost:-nghost]
    
    # Caluculation new solution. One timestep in selected scheme.
    if scheme == 'ftcs':
        u_new_inner = u_inner - 0.5 * nu * (u_right - u_left)
    
    elif scheme == 'upwind':
        u_new_inner = u_inner - nu * (u_inner - u_left)
        
    elif scheme == 'lf':
        u_new_inner = 0.5 * (u_right + u_left) - 0.5 * nu * (u_right - u_left)
        
    elif scheme == 'lw':
        u_new_inner = u_inner - 0.5 * nu * (u_right - u_left) + 0.5 * nu * nu * (u_right - 2 * u_inner + u_left)
        
    else:
        print('Error: unknown scheme. Exiting.')
        exit()

    # Create a copy so that we can still use a u_new_inner
    u_inner = u_new_inner.copy()
    
    # Expand u
    u = np.concatenate(([u_inner[0]],u_inner,[u_inner[-1]]))

    # Update time
    t = t + dt
    nstep = nstep + 1
    
    # 1/0
    if nstep % n_output == 0:
        u_inner.tofile(f'{out_dir}/{scheme}-{int(nstep/n_output):0>4d}.dat')
       